In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
import cv2
import scipy.misc


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


TensorFlow Version: 1.2.1
Default GPU Device: /gpu:0


In [ ]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out

# tests.test_load_vgg(load_vgg, tf)
print("**")

In [ ]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    conv1x1= tf.layers.conv2d(vgg_layer7_out, num_classes, kernel_size=(1,1), strides=(1,1), padding='SAME', kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    layer4_decoder = tf.layers.conv2d_transpose(conv1x1, num_classes, kernel_size=(4,4), strides=(2,2), padding='SAME',\
                                       kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    vgg_layer4_out = tf.layers.conv2d(vgg_layer4_out, num_classes, kernel_size=(1,1), padding='SAME', kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    layer4_decoder = tf.add(layer4_decoder, vgg_layer4_out)
    
    layer3_decoder = tf.layers.conv2d_transpose(layer4_decoder, num_classes, kernel_size=(4,4), strides=(2,2), padding='SAME',\
                                       kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    vgg_layer3_out = tf.layers.conv2d(vgg_layer3_out, num_classes, kernel_size=(1,1), padding='SAME', kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    layer3_decoder = tf.add(layer3_decoder, vgg_layer3_out)
    
    output = tf.layers.conv2d_transpose(layer3_decoder, num_classes, kernel_size=(16,16), strides=(8,8), padding='SAME', \
                                       kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    return output
# tests.test_layers(layers)

In [ ]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    correct_label = tf.reshape(correct_label, (-1,num_classes))
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label))
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss) 

    return logits, train_op, cross_entropy_loss
# tests.test_optimize(optimize)

In [ ]:
def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    if conv == 'HSV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    if conv == 'HLS':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    if conv == 'YUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    if conv == 'BGR':
        return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

In [ ]:
import numpy as np
image_shape = (160, 576)
image_file = "./data/data_road/testing/image_2/um_000001.png"
image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
flip_img = np.flip(image, axis=1)
# image = convert_color(image, "HLS")
import matplotlib.pyplot as plt
plt.imshow(image)
plt.show()
plt.imshow(flip_img)
plt.show()

In [ ]:
from glob import glob
import numpy as np
import shutil
import random
import re
batch_size = 32
data_folder = './data/data_road/training'
image_paths = glob(os.path.join(data_folder, 'image_2', '*.png'))
label_paths = {
    re.sub(r'_(lane|road)_', '_', os.path.basename(path)): path
    for path in glob(os.path.join(data_folder, 'gt_image_2', '*_road_*.png'))}
background_color = np.array([255, 0, 0])
other_road_color = np.array([0,0,0])
random.shuffle(image_paths)
print(len(image_paths))
print()
for batch_i in range(0, len(image_paths), batch_size):
    images = []
    gt_images = []
    #print(batch_i, " , ",batch_i, " : ",batch_i+batch_size ,",", len(image_paths[batch_i:batch_i+batch_size]))
    for image_file in image_paths[batch_i:batch_i+batch_size]:
        image_file ="./data/data_road/training/image_2/um_000009.png"
        gt_image_file = label_paths[os.path.basename(image_file)]

        image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
#         image = convert_color(image)
        plt.imshow(image)
        plt.show()
        gt_image = scipy.misc.imresize(scipy.misc.imread(gt_image_file), image_shape)

        gt_image_flip = np.flip(gt_image, axis=1)
        print("ground truth")
        plt.imshow(gt_image)
        plt.show()
        plt.imshow(gt_image_flip)
        plt.show()
        
        gt_bg = np.all(gt_image == background_color, axis=2)
        gt_bg_flip = np.all(gt_image_flip == background_color, axis=2)
        print("background")
        plt.imshow(gt_bg, cmap="gray")
        plt.show()
        plt.imshow(gt_bg_flip, cmap="gray")
        plt.show()
        
        print("road")
        r_mask = ((gt_image == other_road_color) | (gt_image == background_color))
        r_mask_flip = ((gt_image_flip == other_road_color) | (gt_image_flip == background_color))
        gt_r = np.invert(np.all(r_mask, axis=2))
        gt_r_flip = np.invert(np.all(r_mask_flip, axis=2))
        plt.imshow(gt_r, cmap="gray")
        plt.show()
        plt.imshow(gt_r_flip, cmap="gray")
        plt.show()
        
        
        print("other_road")
        or_mask = (gt_image == other_road_color)
        or_mask_flip = (gt_image_flip == other_road_color)
        gt_or = np.all(or_mask, axis=2)
        gt_or_flip = np.all(or_mask_flip, axis=2)
        plt.imshow(gt_or, cmap="gray")
        plt.show()
        plt.imshow(gt_or_flip, cmap="gray")
        plt.show()
       
        gt_bg = gt_bg.reshape(*gt_bg.shape, 1)
        gt_or = gt_or.reshape(*gt_or.shape, 1)
        gt_r = gt_r.reshape(*gt_r.shape, 1)
        gt_image = np.concatenate((gt_bg, gt_r,gt_or), axis=2)
        print(gt_image)
        print(X)

In [ ]:

print("**")
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    sess.run(tf.global_variables_initializer())
    # TODO: Implement function
    for epoch in range(0, epochs):
        for train_images, train_labels in get_batches_fn(batch_size):
            sess.run(train_op, feed_dict={correct_label: train_labels, 
                                          input_image: train_images, 
                                          learning_rate: 0.001,
                                          keep_prob: 0.5})
    
# tests.test_train_nn(train_nn)

In [ ]:

def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)
    batch_size = 32
    epochs = 10
    
    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/
    
    
    with tf.Session() as sess:
        tf.global_variables_initializer()
        #tf.local_variables_initializer()
        
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        input_image, keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out = load_vgg(sess, vgg_path)
        nn_last_layer = layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes)
        
        learning_rate = tf.placeholder(dtype=tf.float32)
        correct_label = tf.placeholder(dtype=tf.int32, shape=[None, None, None, num_classes])
        
        logits, train_op, cross_entropy_loss = optimize(nn_last_layer,  correct_label, learning_rate, num_classes)
        
        # TODO: Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,\
                 correct_label, keep_prob, learning_rate)
        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

        # OPTIONAL: Apply the trained model to a video



In [ ]:
run()

In [6]:
import scipy.misc
import numpy as np

image_shape = (160, 576)

def process_image(image):
#     print(type(image), " ", image.shape)
#     image = np.resize(image,image_shape)
    image = scipy.misc.imresize(image, image_shape)
    session = tf.get_default_session()
    
    im_softmax = sess.run(
        [tf.nn.softmax(logits)],
        {keep_prob: 1.0, image_input: [image]})
    im_softmax_road = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
    segmentation_road = (im_softmax_road > 0.5).reshape(image_shape[0], image_shape[1], 1)

    im_softmax_other_road = im_softmax[0][:, 2].reshape(image_shape[0], image_shape[1])
    segmentation_other_road = (im_softmax_other_road > 0.5).reshape(image_shape[0], image_shape[1], 1)

    mask_road = np.dot(segmentation_road, np.array([[0, 255, 0, 127]]))
    mask_road = scipy.misc.toimage(mask_road, mode="RGBA")

    mask_other_road = np.dot(segmentation_other_road, np.array([[0, 0, 255, 127]]))
    mask_other_road = scipy.misc.toimage(mask_other_road, mode="RGBA")

    street_im = scipy.misc.toimage(image)
    street_im.paste(mask_road, box=None, mask=mask_road)
    street_im.paste(mask_other_road, box=None, mask=mask_other_road)

    
    
    

    return np.array(street_im)

In [4]:
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [7]:
video_output = 'output.mp4'
clip = VideoFileClip('./video/project_video.mp4')

with tf.Session() as sess:
    
    saver = tf.train.import_meta_graph('mymodel_7.ckpt.meta')
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    graph = tf.get_default_graph()
    logits = graph.get_tensor_by_name("logits:0")
    keep_prob = graph.get_tensor_by_name("keep_prob:0")
    image_input = graph.get_tensor_by_name("image_input:0")
#     image_file = "./data/data_road/training/image_2/um_000012.png"
#     image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
#     im = process_image(image)
   
    project_clip = clip.fl_image(process_image)
    %time project_clip.write_videofile(video_output, audio=False)

INFO:tensorflow:Restoring parameters from ./mymodel_7.ckpt
[MoviePy] >>>> Building video output.mp4
[MoviePy] Writing video output.mp4


100%|█████████▉| 1260/1261 [23:31<00:01,  1.17s/it]

[MoviePy] Done.
[MoviePy] >>>> Video ready: output.mp4 

CPU times: user 14min 52s, sys: 9min 10s, total: 24min 3s
Wall time: 23min 31s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))